In [26]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
from matplotlib import pyplot as plt
%config InlineBackend.figure_format = 'retina'

In [38]:
import pandas as pd
data = pd.read_csv('features_30_sec.csv')
# print(data.info())

#label is categorical data so converting it to numerical
data['label'] = pd.factorize(data['label'])[0]

y = data['label']
# print(data['label'])
data.drop(['filename', 'label'], axis=1, inplace=True)
# print(data.head())
#checking for missing data
missing=data.isnull()
# print(missing.describe())


# KNN

In [3]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

#splitting dataset before applying KNN
X_train, X_holdout, y_train, y_holdout = train_test_split(data.values, y, test_size=0.2,random_state=17)

knn = KNeighborsClassifier(n_neighbors=5)

# for kNN, we need to scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [31]:
from sklearn.metrics import accuracy_score
knn_pred = knn.predict(X_holdout_scaled)
print (f" Accuracy for k-nn is {accuracy_score(y_holdout, knn_pred)}")

 Accuracy for k-nn is 0.665


# Tunning KNN

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score

knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])

knn_params = {'knn__n_neighbors': range(1, 10)}

#cross validation using 5 folds
knn_grid = GridSearchCV(knn_pipe, knn_params,cv=5, n_jobs=-1, verbose=True)

knn_grid.fit(X_train, y_train)

print(knn_grid.best_params_, knn_grid.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
{'knn__n_neighbors': 1} 0.6625


In [33]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score

knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])

knn_params = {'knn__n_neighbors': range(1, 10)}

knn_grid = GridSearchCV(knn_pipe, knn_params,
                        cv=10, n_jobs=-1, verbose=True)

knn_grid.fit(X_train, y_train)

print(knn_grid.best_params_, knn_grid.best_score_)

Fitting 10 folds for each of 9 candidates, totalling 90 fits
{'knn__n_neighbors': 1} 0.67625


In [36]:
accuracy_score(y_holdout, knn_grid.predict(X_holdout)) 
print('train accuaracy=',accuracy_score(y_train, knn_grid.predict(X_train)) )
print('test accuaracy=',accuracy_score(y_holdout, knn_grid.predict(X_holdout)) )

train accuaracy= 0.99875
test accuaracy= 0.695


# F1 Score

In [20]:
from sklearn.metrics import f1_score

f1_score(y_holdout, knn_grid.predict(X_holdout), average='macro')

0.6640616232212999

# AUC

In [37]:
from sklearn.metrics import roc_auc_score

# roc_auc_score(y_holdout, knn_grid.predict(X_holdout),average='macro',multi_class='ovr')

# SVM